## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [2]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

Using TensorFlow backend.
C:\Users\cerbe\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\cerbe\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\cerbe\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\cerbe\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [3]:
#超參數
batch_size = 256 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
num_epochs = 20 # 訓練的 epochs 數量

In [4]:
#資料處理
data_generator = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# x_train = next(data_generator.flow(x_train, shuffle=False))

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
#建立模型
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
______________________

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),
                    samples_per_epoch=x_train.shape[0],
                    steps_per_epoch= x_train.shape[0]//batch_size,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    initial_epoch = 0,
                    epochs = num_epochs) 

# model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),
#                     samples_per_epoch=x_train.shape[0],
#                     steps_per_epoch= 390,
#                     verbose=1, 
#                     validation_data=(x_test, y_test),
#                     initial_epoch = 0,
#                     epochs = epochs) 

#                     validation_data=data_generator.flow(x_test, y_test, batch_size=batch_size),
#                     validation_steps=x_test.shape[0]//batch_size,


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\cerbe\anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., steps_per_epoch=195, verbose=1, validation_data=(array([[[..., initial_epoch=0, epochs=20)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/20
195/195 [==============================] - 17s 85ms/step - loss: 2.0612 - acc: 0.2419 - val_loss: 1.8879 - val_acc: 0.3138
Epoch 2/20
195/195 [==============================] - 12s 59ms/step - loss: 1.7997 - acc: 0.3489 - val_loss: 1.4409 - val_acc: 0.4813
Epoch 3/20
195/195 [==============================] - 12s 59ms/step - loss: 1.6631 - acc: 0.3961 - val_loss: 1.3754 - val_acc: 0.5042
Epoch 4/20
195/195 [==============================] - 11s 59ms/step - loss: 1.5679 - acc: 0.4314 - val_loss: 1.3100 - val_acc: 0.5311
Epoch 5/20
195/195 [==============================] - 11s 58ms/step - loss: 1.5034 - acc: 0.4584 - val_loss: 1.2060 - val_acc: 0.5718
Epoch 6/20
195/195 [==============================] - 11s 59ms/step - loss: 1.4451 - acc: 0.4823 - val_loss: 1.2921 - val_acc: 0.5432
Epoch 7/20
195/195 [==============================] - 12s 59ms/step - loss: 1.4043 - acc: 0.4963 - val_loss: 1.1168 - val_acc: 0.5966
Epoch 8/20
195/195 [==============================] - 12s 59ms